Problem Statement 1.
Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
Rename the columns as per the description from this file:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Task:
Create a sql db from adult dataset and name it sqladb
1. Select 10 records from the adult sqladb
2. Show me the average hours per week of all men who are working in private sector
3. Show me the frequency table for education, occupation and relationship, separately
4. Are there any people who are married, working in private sector and having a masters
degree
5. What is the average, minimum and maximum age group for people working in
different sectors
6. Calculate age distribution by country
7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and
'capital-loss'

In [1]:
# import the library SQLite
import sqlite3

# Load our regular libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

#### Read the dataset and display the first few columns to see what we have to work with

In [2]:
adult_data_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')
#adult_data_df = pd.read_csv('adult.data')
display(adult_data_df.head(3))

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K


#### Rename the columns as assigned
Notice that the final column <=50 etc... did not have a column name assigned in the reference above, so I just assigned it "income" because that seemed the logical thing to do :)


In [3]:
adult_data_df.columns =["age","workclass","fnlwgt","education","education_num","marital_status","occupation","relationship","race","sex","capital_gain","capital_loss","hours_per_week","native_country","income"]

#### Display the changes

In [4]:
display(adult_data_df.head(3))

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K


#### Start your Engine with sqlalchemy! :)

Notice that in the create_engine statement echo is set to 'True', this would NOT be done in a production environment, but is here so that the output can clearly be seen... what "echo=True" does is initiate Python's standard logging module so the output may be easily viewed.

In [5]:
import sqlalchemy

from sqlalchemy import create_engine

engine = create_engine('sqlite:///sqladb', echo=True)

### Convert Pandas DataFrame to sql
Note: Be careful with the "if_exists='replace'" flag below in production databases. Obviously you would not want to overwrite something important! I simply used it here for convenience sake :)

In [6]:
adult_data_df.to_sql('sqladb', engine, if_exists='replace')

2019-07-21 01:21:16,236 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-21 01:21:16,238 INFO sqlalchemy.engine.base.Engine ()
2019-07-21 01:21:16,238 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-21 01:21:16,239 INFO sqlalchemy.engine.base.Engine ()
2019-07-21 01:21:16,240 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("sqladb")
2019-07-21 01:21:16,241 INFO sqlalchemy.engine.base.Engine ()
2019-07-21 01:21:16,244 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("sqladb")
2019-07-21 01:21:16,245 INFO sqlalchemy.engine.base.Engine ()
2019-07-21 01:21:16,248 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-21 01:21:16,249 INFO sqlalchemy.engine.base.Engine ()
2019-07-21 01:21:16,251 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("sqladb")
2019-07-21 01:21:16,251 INFO sqlalchemy.engine.base.Engine ()
2019-0

#### Make a basic connection to the db

In [7]:
connection = sqlite3.connect("sqladb")
cursor = connection.cursor()

#### Return basic column information of the DB


In [8]:
cursor.execute('PRAGMA TABLE_INFO({})'.format("sqladb"))
cursor.fetchall()

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'age', 'BIGINT', 0, None, 0),
 (2, 'workclass', 'TEXT', 0, None, 0),
 (3, 'fnlwgt', 'BIGINT', 0, None, 0),
 (4, 'education', 'TEXT', 0, None, 0),
 (5, 'education_num', 'BIGINT', 0, None, 0),
 (6, 'marital_status', 'TEXT', 0, None, 0),
 (7, 'occupation', 'TEXT', 0, None, 0),
 (8, 'relationship', 'TEXT', 0, None, 0),
 (9, 'race', 'TEXT', 0, None, 0),
 (10, 'sex', 'TEXT', 0, None, 0),
 (11, 'capital_gain', 'BIGINT', 0, None, 0),
 (12, 'capital_loss', 'BIGINT', 0, None, 0),
 (13, 'hours_per_week', 'BIGINT', 0, None, 0),
 (14, 'native_country', 'TEXT', 0, None, 0),
 (15, 'income', 'TEXT', 0, None, 0)]

#### Return total number of rows in DB

In [9]:
cursor.execute('SELECT COUNT(*) FROM {}'.format("sqladb"))
cursor.fetchall()

[(32560,)]

##### Question 1: Select 10 records from the DB

In [10]:
#cursor.execute('SELECT * FROM {tn} limit 10'. format(tn="sqladb"))
#output = cursor.fetchall()
#print(output)
#
# USE Pandas Print because it is cleaner
print(pd.read_sql_query('SELECT * FROM sqladb limit 10', connection))

   index  age          workclass  fnlwgt      education  education_num  \
0      0   50   Self-emp-not-inc   83311      Bachelors             13   
1      1   38            Private  215646        HS-grad              9   
2      2   53            Private  234721           11th              7   
3      3   28            Private  338409      Bachelors             13   
4      4   37            Private  284582        Masters             14   
5      5   49            Private  160187            9th              5   
6      6   52   Self-emp-not-inc  209642        HS-grad              9   
7      7   31            Private   45781        Masters             14   
8      8   42            Private  159449      Bachelors             13   
9      9   37            Private  280464   Some-college             10   

           marital_status          occupation    relationship    race  \
0      Married-civ-spouse     Exec-managerial         Husband   White   
1                Divorced   Handlers-cl

In [11]:
display(adult_data_df.head(3))

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K


##### Question 2: Average hours per week of all men working in the private sector

In [12]:
cursor.execute('SELECT avg(hours_per_week) FROM sqladb WHERE workclass=" Private"')
output = cursor.fetchall()
print(output)

[(40.267095523440254,)]


##### Question 3a: Frequency table for education

In [13]:
cursor.execute('SELECT education, COUNT(*) FROM sqladb GROUP BY education')
output = cursor.fetchall()
display(output)

[(' 10th', 933),
 (' 11th', 1175),
 (' 12th', 433),
 (' 1st-4th', 168),
 (' 5th-6th', 333),
 (' 7th-8th', 646),
 (' 9th', 514),
 (' Assoc-acdm', 1067),
 (' Assoc-voc', 1382),
 (' Bachelors', 5354),
 (' Doctorate', 413),
 (' HS-grad', 10501),
 (' Masters', 1723),
 (' Preschool', 51),
 (' Prof-school', 576),
 (' Some-college', 7291)]

##### Question 3b: Frequency table for occupation

In [14]:
cursor.execute('SELECT occupation, COUNT(*) FROM sqladb GROUP BY occupation')
output = cursor.fetchall()
display(output)

[(' ?', 1843),
 (' Adm-clerical', 3769),
 (' Armed-Forces', 9),
 (' Craft-repair', 4099),
 (' Exec-managerial', 4066),
 (' Farming-fishing', 994),
 (' Handlers-cleaners', 1370),
 (' Machine-op-inspct', 2002),
 (' Other-service', 3295),
 (' Priv-house-serv', 149),
 (' Prof-specialty', 4140),
 (' Protective-serv', 649),
 (' Sales', 3650),
 (' Tech-support', 928),
 (' Transport-moving', 1597)]

##### Question 3c: Frequency table for relationship

In [15]:
cursor.execute('SELECT relationship, COUNT(*) FROM sqladb GROUP BY relationship')
output = cursor.fetchall()
display(output)

[(' Husband', 13193),
 (' Not-in-family', 8304),
 (' Other-relative', 981),
 (' Own-child', 5068),
 (' Unmarried', 3446),
 (' Wife', 1568)]

##### Question 4: Are there any people who are married, working in private sector and having a masters degree

In [16]:
#cursor.execute('SELECT * FROM sqladb WHERE education=" Masters" AND workclass=" Private" AND marital_status=" Married-civ-spouse"')
#output = cursor.fetchall()
#print(output)
#
# USE Pandas Print because it is cleaner
print(pd.read_sql_query('SELECT * FROM sqladb WHERE education=" Masters" AND workclass=" Private" AND marital_status=" Married-civ-spouse"', connection).head(10))

   index  age workclass  fnlwgt education  education_num       marital_status  \
0      4   37   Private  284582   Masters             14   Married-civ-spouse   
1     86   33   Private  202051   Masters             14   Married-civ-spouse   
2     99   76   Private  124191   Masters             14   Married-civ-spouse   
3    187   31   Private   99928   Masters             14   Married-civ-spouse   
4    197   35   Private  138992   Masters             14   Married-civ-spouse   
5    310   34   Private  142897   Masters             14   Married-civ-spouse   
6    360   62   Private  270092   Masters             14   Married-civ-spouse   
7    404   41   Private  445382   Masters             14   Married-civ-spouse   
8    434   33   Private  208405   Masters             14   Married-civ-spouse   
9    467   49   Private  192776   Masters             14   Married-civ-spouse   

         occupation     relationship                 race      sex  \
0   Exec-managerial             Wife  

##### Just making a note for myself..

In [ ]:
cursor.execute('SELECT avg(age) AS average, 
               max(age) AS maximum, 
               min(age) AS minimum 
               FROM sqladb 
               GROUP BY occupation')
output = cursor.fetchall()
display(output)

##### Question 5a: The minimum age for working people by sector

In [17]:
cursor.execute('SELECT occupation, min(age) FROM sqladb GROUP BY occupation')
output = cursor.fetchall()
display(output)

[(' ?', 17),
 (' Adm-clerical', 17),
 (' Armed-Forces', 23),
 (' Craft-repair', 17),
 (' Exec-managerial', 17),
 (' Farming-fishing', 17),
 (' Handlers-cleaners', 17),
 (' Machine-op-inspct', 17),
 (' Other-service', 17),
 (' Priv-house-serv', 17),
 (' Prof-specialty', 17),
 (' Protective-serv', 17),
 (' Sales', 17),
 (' Tech-support', 17),
 (' Transport-moving', 17)]

##### Question 5b: The maximum age for working people by sector

In [18]:
cursor.execute('SELECT occupation, max(age) FROM sqladb GROUP BY occupation')
output = cursor.fetchall()
display(output)

[(' ?', 90),
 (' Adm-clerical', 90),
 (' Armed-Forces', 46),
 (' Craft-repair', 90),
 (' Exec-managerial', 90),
 (' Farming-fishing', 90),
 (' Handlers-cleaners', 90),
 (' Machine-op-inspct', 90),
 (' Other-service', 90),
 (' Priv-house-serv', 81),
 (' Prof-specialty', 90),
 (' Protective-serv', 90),
 (' Sales', 90),
 (' Tech-support', 73),
 (' Transport-moving', 90)]

##### Question 5c: The average age for working people by sector

In [19]:
cursor.execute('SELECT occupation, avg(age) FROM sqladb GROUP BY occupation')
output = cursor.fetchall()
display(output)

[(' ?', 40.882799782962564),
 (' Adm-clerical', 36.96391615813213),
 (' Armed-Forces', 30.22222222222222),
 (' Craft-repair', 39.03147109050988),
 (' Exec-managerial', 42.16920806689621),
 (' Farming-fishing', 41.2112676056338),
 (' Handlers-cleaners', 32.16569343065694),
 (' Machine-op-inspct', 37.71528471528472),
 (' Other-service', 34.94962063732929),
 (' Priv-house-serv', 41.7248322147651),
 (' Prof-specialty', 40.51763285024155),
 (' Protective-serv', 38.9537750385208),
 (' Sales', 37.353972602739724),
 (' Tech-support', 37.022629310344826),
 (' Transport-moving', 40.19787100814026)]

##### Question 6a: Calculate minimum age distribution by country

In [20]:
cursor.execute('SELECT native_country, min(age) FROM sqladb GROUP BY native_country')
output = cursor.fetchall()
display(output)

[(' ?', 17),
 (' Cambodia', 18),
 (' Canada', 17),
 (' China', 22),
 (' Columbia', 18),
 (' Cuba', 21),
 (' Dominican-Republic', 18),
 (' Ecuador', 21),
 (' El-Salvador', 17),
 (' England', 17),
 (' France', 20),
 (' Germany', 18),
 (' Greece', 22),
 (' Guatemala', 19),
 (' Haiti', 17),
 (' Holand-Netherlands', 32),
 (' Honduras', 18),
 (' Hong', 19),
 (' Hungary', 24),
 (' India', 17),
 (' Iran', 22),
 (' Ireland', 23),
 (' Italy', 19),
 (' Jamaica', 18),
 (' Japan', 19),
 (' Laos', 19),
 (' Mexico', 17),
 (' Nicaragua', 19),
 (' Outlying-US(Guam-USVI-etc)', 21),
 (' Peru', 17),
 (' Philippines', 17),
 (' Poland', 17),
 (' Portugal', 19),
 (' Puerto-Rico', 17),
 (' Scotland', 18),
 (' South', 19),
 (' Taiwan', 20),
 (' Thailand', 19),
 (' Trinadad&Tobago', 17),
 (' United-States', 17),
 (' Vietnam', 19),
 (' Yugoslavia', 20)]

##### Question 6b: Calculate maximum age distribution by country

In [21]:
cursor.execute('SELECT native_country, max(age) FROM sqladb GROUP BY native_country')
output = cursor.fetchall()
display(output)

[(' ?', 90),
 (' Cambodia', 65),
 (' Canada', 80),
 (' China', 75),
 (' Columbia', 75),
 (' Cuba', 82),
 (' Dominican-Republic', 78),
 (' Ecuador', 90),
 (' El-Salvador', 79),
 (' England', 90),
 (' France', 64),
 (' Germany', 74),
 (' Greece', 65),
 (' Guatemala', 66),
 (' Haiti', 63),
 (' Holand-Netherlands', 32),
 (' Honduras', 58),
 (' Hong', 60),
 (' Hungary', 81),
 (' India', 61),
 (' Iran', 63),
 (' Ireland', 68),
 (' Italy', 77),
 (' Jamaica', 66),
 (' Japan', 61),
 (' Laos', 56),
 (' Mexico', 81),
 (' Nicaragua', 67),
 (' Outlying-US(Guam-USVI-etc)', 63),
 (' Peru', 69),
 (' Philippines', 90),
 (' Poland', 85),
 (' Portugal', 78),
 (' Puerto-Rico', 90),
 (' Scotland', 62),
 (' South', 90),
 (' Taiwan', 61),
 (' Thailand', 55),
 (' Trinadad&Tobago', 61),
 (' United-States', 90),
 (' Vietnam', 73),
 (' Yugoslavia', 66)]

##### Question 6c: Calculate average age distribution by country

In [22]:
cursor.execute('SELECT native_country, avg(age) FROM sqladb GROUP BY native_country')
output = cursor.fetchall()
display(output)

[(' ?', 38.72555746140652),
 (' Cambodia', 37.78947368421053),
 (' Canada', 42.54545454545455),
 (' China', 42.53333333333333),
 (' Columbia', 39.71186440677966),
 (' Cuba', 45.76842105263158),
 (' Dominican-Republic', 37.72857142857143),
 (' Ecuador', 36.642857142857146),
 (' El-Salvador', 34.132075471698116),
 (' England', 41.15555555555556),
 (' France', 38.96551724137931),
 (' Germany', 39.25547445255474),
 (' Greece', 46.206896551724135),
 (' Guatemala', 32.421875),
 (' Haiti', 38.27272727272727),
 (' Holand-Netherlands', 32.0),
 (' Honduras', 33.84615384615385),
 (' Hong', 33.65),
 (' Hungary', 49.38461538461539),
 (' India', 38.09),
 (' Iran', 39.41860465116279),
 (' Ireland', 36.458333333333336),
 (' Italy', 46.42465753424658),
 (' Jamaica', 35.592592592592595),
 (' Japan', 38.24193548387097),
 (' Laos', 34.72222222222222),
 (' Mexico', 33.29082426127527),
 (' Nicaragua', 33.61764705882353),
 (' Outlying-US(Guam-USVI-etc)', 38.714285714285715),
 (' Peru', 35.25806451612903),
 (

##### Question 7: Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

In [ ]:
cursor.execute('UPDATE sqladb SET net_capital_gain=capital_gain-capital_loss')

#### Use Pandas to print out a decent looking sql query from sqlite

In [23]:
print(pd.read_sql_query("SELECT * FROM sqladb", connection).head(10))

   index  age          workclass  fnlwgt      education  education_num  \
0      0   50   Self-emp-not-inc   83311      Bachelors             13   
1      1   38            Private  215646        HS-grad              9   
2      2   53            Private  234721           11th              7   
3      3   28            Private  338409      Bachelors             13   
4      4   37            Private  284582        Masters             14   
5      5   49            Private  160187            9th              5   
6      6   52   Self-emp-not-inc  209642        HS-grad              9   
7      7   31            Private   45781        Masters             14   
8      8   42            Private  159449      Bachelors             13   
9      9   37            Private  280464   Some-college             10   

           marital_status          occupation    relationship    race  \
0      Married-civ-spouse     Exec-managerial         Husband   White   
1                Divorced   Handlers-cl

In [ ]:
connection.close()